In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#!conda install beautifulsoup4
#!conda install lxml
#!conda install requests

## Importing the required libraries for scraping data ##

In [3]:
from bs4 import BeautifulSoup
import requests
source= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')

## Scraping Data from file using BeautifulSoup ##

In [ ]:
table=soup.find(class_="wikitable sortable")
#print(table)
df = pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing Boroughs with NA values, renaming Postcode to PostalCode and Joining Neighbourhoods with same PostalCode

In [ ]:
#indexre= df[ (df['Borough'] == 'Not assigned') & (df['Neighbourhood'] == 'Not assigned')].index
indexre= df[ (df['Borough'] == 'Not assigned')].index
# Delete these row indexes from dataFrame
df.drop(indexre , inplace=True)
df.rename(columns={'Postcode':'PostalCode'},inplace=True)
df = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()
#df.reset_index(inplace=True)
#df.drop('index',axis=1,inplace=True)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## For NA values for Neighbourhood replacing by Borough Names ##

In [ ]:
df.Neighbourhood = np.where(df.Neighbourhood == 'Not assigned', df.Borough, df.Neighbourhood)
df.shape

(103, 3)

## Using Geospatial Data ##

In [ ]:
!wget -q http://cocl.us/Geospatial_data
csvread= pd.read_csv('Geospatial_data')
csvread.shape
csvread.rename(columns={'Postal Code':'PostalCode'},inplace=True)
#csvread.groupby('PostalCode')

In [ ]:
df= pd.concat([df, csvread], sort=False, axis= 1, join='inner')
df = df.loc[:,~df.columns.duplicated()]